# 🚀 Customize and Deploy `Qwen/Qwen2-Audio-7B-Instruct` on Amazon SageMaker AI
---
In this notebook, we explore **Qwen2-Audio-7B-Instruct**, a 7-billion-parameter audio-language model from Alibaba’s Qwen2-Audio series. You’ll learn how to fine-tune it (or use it out-of-the-box), evaluate its audio + text understanding, and deploy it using SageMaker for voice interaction, audio analysis, and multimodal workflows.

**What is Qwen2-Audio-7B-Instruct?**

Qwen2-Audio-7B-Instruct is the instruction-tuned variant of the Qwen2-Audio-7B model. It can take **audio inputs** (speech, other sound signals) and **text instructions** to perform tasks such as transcription, translation, conversational voice chat, audio analysis, and sound-event detection. It supports two primary audio interaction modes: *voice chat* (audio in, audio/text out conversationally) and *audio analysis* (audio + text instructions to analyze or respond). It is released under the **Apache-2.0 license**.  
🔗 Model card: [Qwen/Qwen2-Audio-7B-Instruct on Hugging Face](https://huggingface.co/Qwen/Qwen2-Audio-7B-Instruct)

---

**Key Specifications**≈

| Feature | Details |
|---|---|
| **Parameters** | ~7 billion |
| **Modalities** | Audio input + Text input → Text output |
| **Interaction Modes** | Voice chat; Audio analysis (audio + instruction) |
| **Context / Window Length** | ~32,768 tokens for text; audio input support for various sampling rates |
| **Languages Supported** | English and Chinese (at minimum) |
| **License** | Apache-2.0 |

---

**Benchmarks & Behavior**

- Qwen2-Audio has been evaluated across tasks like automatic speech recognition (ASR), speech-to-text translation, emotion recognition, and sound event classification, showing state-of-the-art or competitive performance.  
- It is designed to understand complex audio environments (multi-speaker or noisy input) and respond appropriately to natural instructions.  
- Strong fit for applications in **multilingual speech agents, customer service bots, transcription services, and audio analytics pipelines**.  

---

In [ ]:
%pip install -Uq sagemaker "datasets[audio]"

In [ ]:
import boto3
import sagemaker

In [ ]:
region = boto3.Session().region_name

sess = sagemaker.Session(boto3.Session(region_name=region))

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

In [ ]:
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

## Data Preparation for Supervised Fine-tuning

### [AudioSet-Audio-Instructions](https://huggingface.co/datasets/mesolitica/AudioSet-Audio-Instructions)

**AudioSet-Audio-Instructions** is a multimodal dataset created by pairing **Google AudioSet audio clips** with human-written instruction–response examples. It is designed to support **instruction tuning for audio understanding** tasks such as event detection, audio classification, and acoustic reasoning.

**Data Format & Structure**:
- Built on **AudioSet**, a large ontology of over 500 audio event classes.  
- Each example contains:  
  • `audio` — a 10-second audio clip from AudioSet.  
  • `instruction` — a natural language prompt about the audio (e.g., “Identify the main sound in this clip”).  
  • `response` — the assistant’s answer (e.g., “This audio contains the sound of a violin being played”).  
- Distributed in **JSON/Parquet** formats for ease of use.  
- Includes `train`, `validation`, and `test` splits.  

**License**: Same as **AudioSet** (Creative Commons Attribution license) with additional generated annotations shared openly for research.  

**Applications**:

The dataset is useful for training and evaluating multimodal models on:  
- Audio event classification and tagging  
- Instruction-following tasks for sound and acoustic scenes  
- Building conversational audio assistants  
- Enhancing models for **multimodal reasoning** (sound + text)  

---

*This dataset bridges raw audio perception with natural language supervision, making it a valuable resource for instruction-tuned audio understanding models.*  


In [ ]:
import os
import json
import pprint
from tqdm import tqdm
import soundfile as sf
from datasets import load_dataset, Audio

In [ ]:
dataset_parent_path = os.path.join(os.getcwd(), "tmp_cache_local_dataset")
os.makedirs(dataset_parent_path, exist_ok=True)

**Preparing Your Dataset in `messages` format**

This section walks you through creating a conversation-style dataset—the required `messages` format—for directly training LLMs using SageMaker AI.

**What Is the `messages` Format?**

The `messages` format structures instances as chat-like exchanges, wrapping each conversation turn into a role-labeled JSON array. It’s widely used by frameworks like TRL.

Example entry:

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "How do I bake sourdough?" },
    { "role": "assistant", "content": "First, you need to create a starter by..." }
  ]
}


In [ ]:
dataset_name = "mesolitica/AudioSet-Audio-Instructions"
dataset = load_dataset(dataset_name, split="500k_part1_speech[:200]") 
dataset = dataset.cast_column("audio_filename", Audio(sampling_rate=16000))

In [ ]:
dataset_parent_path = os.path.join(os.getcwd(), "tmp_cache_local_dataset")
os.makedirs(dataset_parent_path, exist_ok=True)
audio_dir = os.path.join(dataset_parent_path, "audio_files")

# final dataset file path
dataset_filename = os.path.join(
    dataset_parent_path,
    f"{dataset_name.replace('/', '--').replace('.', '-')}.jsonl"
)

In [ ]:
pprint.pp(dataset[0])

In [ ]:
print(f"total number of fine-tunable samples: {len(dataset)}")

In [ ]:
def convert_to_messages_audio(row, idx):
    os.makedirs(audio_dir, exist_ok=True)

    # decode audio
    audio = row["audio_filename"]
    array, sr = audio["array"], audio["sampling_rate"]

    # choose filename
    key = f"sample-{idx:06d}"
    filename = f"{key}.wav"
    filepath = os.path.join(audio_dir, filename)

    # save wav
    sf.write(filepath, array, sr)

    # system instruction
    system_content = (
        "You are an audio understanding assistant. Listen carefully to the audio clip, "
        "analyze the sounds, and provide a clear and concise description."
    )

    # messages schema (all content fields are lists of dicts)
    return {
        "messages": [
            {
                "role": "system",
                "content": [
                    {"type": "text", "text": system_content}
                ],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "audio",
                        "audio_url": f"file:///opt/ml/input/data/training/{os.path.relpath(filepath, dataset_parent_path)}"
                    },
                    {"type": "text", "text": row["question"]},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": row["answer"]}
                ],
            },
        ]
    }


with open(dataset_filename, "w") as f: 
    for idx, row in tqdm(enumerate(dataset), total=len(dataset)): 
        processed = convert_to_messages_audio(row, idx) 
        f.write(json.dumps(processed) + "\n") 
        
print(f"✅ Saved JSONL dataset to {dataset_filename}") 
print(f"✅ Audio files saved under {audio_dir}")

#### Upload file to S3

In [ ]:
from sagemaker.s3 import S3Uploader

In [ ]:
data_s3_uri = f"s3://{sess.default_bucket()}/dataset"

uploaded_s3_uri = S3Uploader.upload(
    local_path=dataset_filename,
    desired_s3_uri=data_s3_uri
)
print(f"Uploaded {dataset_filename} to > {uploaded_s3_uri}")

In [ ]:
data_s3_uri = f"s3://{sess.default_bucket()}/dataset"

uploaded_audio_s3_uri = S3Uploader.upload(
    local_path=os.path.join(os.path.dirname(dataset_filename), "audio_files"),
    desired_s3_uri=data_s3_uri
)

print(f"Uploaded {dataset_filename} to > {uploaded_audio_s3_uri}")

## Fine-Tune LLMs using SageMaker `Estimator`/`ModelTrainer`

In [ ]:
import time
from sagemaker.modules.configs import (
    CheckpointConfig,
    Compute,
    OutputDataConfig,
    SourceCode,
    StoppingCondition,
)
from sagemaker.modules.configs import InputData
from sagemaker.modules.train import ModelTrainer
from getpass import getpass
import yaml
from jinja2 import Template

In [ ]:
MODEL_ID = "Qwen/Qwen2-Audio-7B-Instruct"

In [ ]:
hf_token = getpass()

### Training using `PyTorch` `ModelTrainer`
---
**Observability**: SageMaker AI has [SageMaker MLflow](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html) which enables you to accelerate generative AI by making it easier to track experiments and monitor performance of models and AI applications using a single tool.

You can choose to include MLflow as a part of your training workflow to track your model fine-tuning metrics in realtime by simply specifying a **mlflow** tracking arn.

Optionally you can also report to : **tensorboard**, **wandb**.

In [ ]:
MLFLOW_TRACKING_SERVER_ARN = "arn:aws:sagemaker:us-east-1:811828458885:mlflow-tracking-server/mlflow-demos"

if MLFLOW_TRACKING_SERVER_ARN:
    reports_to = "mlflow"
else:
    reports_to = "tensorboard"

In [ ]:
job_name = MODEL_ID.replace('/', '--').replace('.', '-')

In [ ]:
if MLFLOW_TRACKING_SERVER_ARN:
    training_env = {
        # mlflow tracking metrics
        "MLFLOW_EXPERIMENT_NAME": f"{job_name}-exp",
        "MLFLOW_TAGS": json.dumps(
            {
                "source.job": "sm-training-jobs", 
                "source.type": "sft", 
                "source.framework": "pytorch"
            }
        ),
        "MLFLOW_TRACKING_URI": MLFLOW_TRACKING_SERVER_ARN,
        "MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING": "true",
        # non tracking metrics - enabled
        "HF_TOKEN": hf_token,
        "FI_EFA_USE_DEVICE_RDMA": "1",
        "NCCL_DEBUG": "INFO",
        "NCCL_SOCKET_IFNAME": "eth0",
        "FI_PROVIDER": "efa",
        "NCCL_PROTO": "simple",
        "NCCL_NET_GDR_LEVEL": "5"
    }
else:
    training_env = {
        # non tracking metrics
        "HF_TOKEN": hf_token,
        "FI_EFA_USE_DEVICE_RDMA": "1",
        "NCCL_DEBUG": "INFO",
        "NCCL_SOCKET_IFNAME": "eth0",
        "FI_PROVIDER": "efa",
        "NCCL_PROTO": "simple",
        "NCCL_NET_GDR_LEVEL": "5"
    }

#### Training strategy - Choose between: `PeFT`/`Spectrum`/`Full-Finetuning`

Here we create a measured mapping of strategy to instance.

In [ ]:
%%writefile sagemaker_code/requirements.txt
transformers==4.57.0
peft==0.17.0
accelerate==1.11.0
bitsandbytes==0.46.1
datasets==4.0.0
deepspeed==0.17.5
hf-transfer==0.1.8
hf_xet
liger-kernel==0.6.1
lm-eval[api]==0.4.9
kernels>=0.9.0
mlflow
Pillow
safetensors>=0.6.2
sagemaker==2.251.1
sagemaker-mlflow==0.1.0
sentencepiece==0.2.0
tokenizers>=0.21.4
triton
trl==0.21.0
tensorboard
psutil
py7zr
git+https://github.com/triton-lang/triton.git@main#subdirectory=python/triton_kernels
vllm==0.10.1
poetry
yq
psutil
nvidia-ml-py
pyrsmi

In [ ]:
# For PeFT
args = [
    "--config",
    "hf_recipes/Qwen/Qwen2-Audio-7B-Instruct-vanilla-peft-qlora.yaml",
    # "--run-eval" # enable this for small models to run eval + tune
]
training_instance_type = "ml.g6e.2xlarge"
training_instance_count = 1

## For Full-Finetuning
# args = [
#     "--config",
#     "hf_recipes/Qwen/Qwen2-Audio-7B-Instruct-vanilla-full.yaml",
#     # "--run-eval" # enable this for small models if you're looking to bundle eval with fine-tuning
# ]
# training_instance_type = "ml.g6e.4xlarge"
# training_instance_count = 1


In [ ]:
pytorch_image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sess.boto_session.region_name,
    version="2.7.1",
    instance_type=training_instance_type,
    image_scope="training",
)
print(f"Using image: {pytorch_image_uri}")

In [ ]:
source_code = SourceCode(
    source_dir="./sagemaker_code",
    command=f"bash sm_accelerate_train.sh {' '.join(args)}",
)

compute_configs = Compute(
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    keep_alive_period_in_seconds=1800,
    volume_size_in_gb=300
)

base_job_name = f"{job_name}-finetune"
output_path = f"s3://{sess.default_bucket()}/{base_job_name}"

model_trainer = ModelTrainer(
    training_image=pytorch_image_uri,
    source_code=source_code,
    base_job_name=base_job_name,
    compute=compute_configs,
    stopping_condition=StoppingCondition(max_runtime_in_seconds=18000),
    output_data_config=OutputDataConfig(
        s3_output_path=output_path,
    ),
    checkpoint_config=CheckpointConfig(
        s3_uri=output_path + dataset_name.replace('/', '--').replace('.', '-') + "/checkpoints", local_path="/opt/ml/checkpoints"
    ),
    role=role,
    environment=training_env
)

In [ ]:
model_trainer.train(
    input_data_config=[
        InputData(
            channel_name="training",
            data_source=uploaded_s3_uri,  
        )
    ], 
    wait=False
)